In [1]:
import phconvert as phc
# Special thanks to Christoph Gohlke <http://www.lfd.uci.edu/~gohlke/>
from fcsfiles import ConfoCor3Raw
import numpy as np
import glob

### Importing and preparing confocor raw files

In [2]:
ch1raw=ConfoCor3Raw('DATA/mono_prox_8%_Ch1.raw')
ch2raw=ConfoCor3Raw('DATA/mono_prox_8%_Ch2.raw')

ch1raw._fh.seek(128)
# Interphoton delays, timeunit is 1.0/ch1raw.frequency
times = np.fromfile(ch1raw._fh, dtype='<u4', count=-1)
# converting from delays to arrival times
times_acceptor = np.cumsum(times.astype('u8'))

ch2raw._fh.seek(128)
times = np.fromfile(ch2raw._fh, dtype='<u4', count=-1)
times_donor = np.cumsum(times.astype('u8'))

# Creating 2d adday with boolean mask for acceptor
# 1 3 5 7 2 4 6  - arrival times
# 0 0 0 0 1 1 1  - acceptor mask
df = np.hstack([np.vstack( [   times_donor, np.zeros(   times_donor.size)] ),
                np.vstack( [times_acceptor,  np.ones(times_acceptor.size)] )]).T

# sorting photons
# [1,2,3,4,5,6,7] - timestamps
# [0,1,0,1,0,1,0] - acceptor mask
df_sorted = df[np.argsort(df[:,0])].T

In [6]:
timestamps=df_sorted[0].astype('int64')
timestamps_unit = 1.0/ch1raw.frequency

#mask, donor - 0, acceptor - 1, not boolean as there can be multiple detectors
detectors=df_sorted[1].astype('uint8') 

description = 'Super description'
author = 'best author'
author_affiliation = 'Moscow State Uni'

sample_name = 'super sample'
buffer_name = 'super buffer'
dye_names = 'Cy3, Cy5'   # Comma separates names of fluorophores




photon_data = dict(
    timestamps=timestamps,
    detectors=detectors,
    timestamps_specs={'timestamps_unit': timestamps_unit})

setup = dict(
    ## Mandatory fields
    num_pixels = 2,                   # using 2 detectors
    num_spots = 1,                    # a single confoca excitation
    num_spectral_ch = 2,              # donor and acceptor detection 
    num_polarization_ch = 1,          # no polarization selection 
    num_split_ch = 1,                 # no beam splitter
    modulated_excitation = False,     # CW excitation, no modulation 
    excitation_alternated = [False],  # CW excitation, no modulation 
    lifetime = False,                 # no TCSPC in detection
    
    ## Optional fields
    excitation_wavelengths = [532e-9],         # List of excitation wavelenghts
    excitation_cw = [True],                    # List of booleans, True if wavelength is CW
    detection_wavelengths = [580e-9, 640e-9],  # Nominal center wavelength 
                                               # each for detection ch
)

# can be skiped
identity = dict(
    author=author,
    author_affiliation=author_affiliation)

measurement_specs = dict(
    measurement_type = 'smFRET',
    detectors_specs = {'spectral_ch1': [0],  # list of donor's detector IDs
                       'spectral_ch2': [1]}  # list of acceptor's detector IDs
    )

photon_data['measurement_specs'] = measurement_specs

data = dict(
    description=description,
    photon_data = photon_data,
    setup=setup,
    identity=identity
)
phc.hdf5.save_photon_hdf5(data, h5_fname='some_data.h5', overwrite=True)

Saving: mononucleosome.h5


### useful function for bulk convertion
Zeiss Confocor files contain unique experiment ID

This function reads list of input files and returns dictionary:

{'experimental id' : {0:'channel1 filename', 1:'channel2 filename'}...}

In [25]:
def find_paired_Confocorfiles(filenames):
    ids={}
    for name in filenames:
        t=ConfoCor3Raw(name)
        if not (t.measurement_identifier in ids):
            ids[t.measurement_identifier]={t.channel:name}
        else:
            ids[t.measurement_identifier] [t.channel]=name
    for exp_id in ids.items():
        if len(exp_id[1]) != 2:
            ids.pop(exp_id[0])
    return ids

namelist=glob.glob("DATA/*.raw")
print find_paired_Confocorfiles(namelist)

{'15e04f844a12265e6359ba3cb2f7a3d': {0: 'DATA/di_prox_8%_Ch1.raw', 1: 'DATA/di_prox_8%_Ch2.raw'}, '6d05146b4043d2a55d3f0daa5d6c810c': {0: 'DATA/di_prox_20%_1_Ch1.raw', 1: 'DATA/di_prox_20%_1_Ch2.raw'}, '5303c85644f5a8469a4ee9af6531c997': {0: 'DATA/di_prox_20%_2_Ch1.raw', 1: 'DATA/di_prox_20%_2_Ch2.raw'}, 'd7d5b0804c999f93a143d689d930040c': {0: 'DATA/mono_prox_40%_2_Ch1.raw', 1: 'DATA/mono_prox_40%_2_Ch2.raw'}, 'cdf51d6c4aef5eaadeaf5586bc4b8645': {0: 'DATA/di_prox_40%_2_Ch1.raw', 1: 'DATA/di_prox_40%_2_Ch2.raw'}, '48513d74d0223884e071ea64586dd55': {0: 'DATA/bckgrnd_40%_Ch1.raw', 1: 'DATA/bckgrnd_40%_Ch2.raw'}, 'd59147244411113f36c6fe84bee1d2f': {0: 'DATA/mono_prox_8%_Ch1.raw', 1: 'DATA/mono_prox_8%_Ch2.raw'}, '667977364cf5d896857bd9b717242a4d': {0: 'DATA/di_prox_40%_1_Ch1.raw', 1: 'DATA/di_prox_40%_1_Ch2.raw'}, 'f39d44549e2e450efe1b8a17aa411f4': {0: 'DATA/mono_prox_40%_1_Ch1.raw', 1: 'DATA/mono_prox_40%_1_Ch2.raw'}}
